# Using Wolfram Engine in Raku

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
December 2022

## Introduction

In this notebook we are going to demonstrate how to connect to [Wolfram Engine (WE)](https://www.wolfram.com/engine/) using [ZMQ](https://zeromq.org/).

See the related package ["Proc::ZMQed"](https://raku.land/zef:antononcube/Proc::ZMQed), blog post ["Proc::ZMQed"](https://rakuforprediction.wordpress.com/2022/11/29/proczmqed/).

**Remark:** WE uses Wolfam Language (WL). We can say that WE and WL are part of Mathematica. Mathematica and WL are used as synonyms in this notebook.

--------

## Estalishing ZMQ connection

First we have to establish the a connection between WE and our Raku session. We use ["Proc::ZMQed"](https://raku.land/zef:antononcube/Proc::ZMQed).

In [1]:
use Proc::ZMQed::Mathematica;
my Proc::ZMQed::Mathematica $wlProc .= new(url => 'tcp://127.0.0.1', port => '5550');

Proc::ZMQed::Mathematica.new(cli-name => "wolframscript", code-option => "-code", url => "tcp://127.0.0.1", port => "5550", proc => Any, context => Net::ZMQ4::Context, receiver => Net::ZMQ4::Socket)

In [2]:
$wlProc.start-proc():!proclaim;

()

Here is a simple symobolic expansion command (a non-trivial computation to convince ourselves that the connection with WE is working):

In [3]:
my $cmd = 'Expand[(x+y)^4]';
my $wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n $wlRes";

Sent : Expand[(x+y)^4]
Got  :
  4      3        2  2        3    4
x  + 4 x  y + 6 x  y  + 4 x y  + y


The result is pretty formated text. Let us get an expression that we can use in Raku:

In [4]:
$cmd = 'FortranForm[Expand[($x+$y)^4]]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  : $wlRes";

Sent : FortranForm[Expand[($x+$y)^4]]
Got  : $x**4 + 4*$x**3*$y + 6*$x**2*$y**2 + 4*$x*$y**3 + $y**4


Her we assign values to the variables in the expression above and evaluate the string with `EVAL`:

In [5]:
my $x = 5;
my $y = 3;

use MONKEY-SEE-NO-EVAL;
say "Using : {{:$x, :$y}.raku}";
say 'EVAL($wlRes) : ', EVAL($wlRes);

Using : {:x(5), :y(3)}
EVAL($wlRes) : 4096


Here is a diagram that summarizes the computations in this section:

![WEandRaku](https://rakuforprediction.files.wordpress.com/2022/11/mermaid-diagram-2022-12-06-092042.png?w=600)

-------

## More advanced computations

Let us look into more advanced computations.

Here we differentiate an expression:

In [6]:
$cmd = 'D[(x+y)^3, x]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n $wlRes";

Sent : D[(x+y)^3, x]
Got  :
          2
3 (x + y)


Here we find the solutions of a differential equation -- note that we have escape `'` when specifying the derivatives:

In [7]:
$cmd = 'sol = DSolve[{y\'[x] == -3 y[x]^2, y[1] == 2}, y[x], x]';

$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n {$wlRes}";

Sent : sol = DSolve[{y'[x] == -3 y[x]^2, y[1] == 2}, y[x], x]
Got  :
              2
{{y[x] -> --------}}
          -5 + 6 x


In [8]:
$cmd='p=Plot[y[x] /. sol, {x, 1, 4}]; ps=ExportString[p,{"Base64","PNG"}]; Export["/tmp/wlimg.txt",ps]';
$wlRes = $wlProc.evaluate($cmd);

/tmp/wlimg.txt

In [9]:
use Text::Plot;
my $timg=slurp('/tmp/wlimg.txt');
from-base64($timg)

<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAWgAAADeCAIAAACfYw1eAAAA1HpUWHRSYXcgcHJvZmlsZSB0eXBl
IGV4aWYAAHjadY/ZiQQxDET/HcWGoMuSHY572g2TwYa/5aNhe2AKLD0XOlDqv+8r/QwJU7Icxas7
QVatSgMUWlqZqc445WUTP/109gXcYOV/Dbd/PP3wlaV8DCJZoGMDWG037EEqy+e2/seul1bKY9B1
XFP7GLL7LP7y1xDPzmGIJhThFVwSEMmzUA6//OWOQvZTyF7I3SPQ5jEqtI3GIenxrO7JFcY5TtDx
RJvyjKLYBzZwVp1+yImqDCYd5/wB+YFcpU9d444AAAAJcEhZcwAACxMAAAsTAQCanBgAAAA8dEVY
dFNvZnR3YXJlAENyZWF0ZWQgd2l0aCB0aGUgV29sZnJhbSBMYW5ndWFnZSA6IHd3dy53b2xmcmFt
LmNvbVyipoUAAAAhdEVYdENyZWF0aW9uIFRpbWUAMjAyMjoxMjoyNCAxNDo1MzoxNyuk4pEAACAA
SURBVHic7d17XFTnuS/wZ839PsNcYC4ww03kJqAC3m8x3VpjTRvUne3OjhpPxJ6SxnQ35GKb9iQ9
OWmbnZNuY05sqon6SU2ssYmpNlGjSEAxgiKiXFSQ4Q7DzDDMDHNf54+hI0WNgjBrZni+f/gZ1juX
5zMufqz3Xe96F0GSJIxGYWFh4PHOnTtH9VqEUGRgjOE1mBcITXI0qgtACIUfDA6E0KhhcCCERg2D
AyE0ahgcCKFRuxUcLpfr9maSJEd7vhYhFPGGguPAgQOLFi0a0bZ///7k5GSlUrlr166gF4YQCl00
APj973//xBNPjDiyMJvNzz333JEjR44dO/b8889bLBaKKkQIhRwaADQ0NOzbt29EQ01NTUZGRmpq
anZ29owZMyoqKh7wk0iStA26H/BNEEKhgAEAf/rTn7799tsRDXq9XqFQ+B8rFIru7u5A09hmnb+2
+6zH63t18/wHqhchFALuOuVcJBLZ7Xb/Y5vNFh0dHWga25RzhYRb1dDtI0kaQYzh5Qih0HGH07Em
k8lqtU6fPv38+fMGg8FgMJw/fz4zM/MBPykuRuT2+Lr7bA/4Pgghyg0FBzHsKKCoqGjHjh1xcXG/
/vWvFy5cmJOT8+KLL2o0mgf8pLgYIQC09gw84PsghChHfPc0Dbfb7fP52Gx2YEthYeHYuipGy+Bz
b59atSCpYMnUMbwcIRQ67nFZPZPJHK9Pkoq4fA5T341HHAiFvaBOOY+NEbZ243wQhMJeUINDGyPs
63fYHTibA6HwFtTgiIsRAUAbjo8iFOaCGxzRQgBoxWEOhMJcUINDEy0kAHCYA6FwN5bFigNTzkd7
XpbNpMfI+C0YHAiFuWCvcq5Tii40dPt8JI2GE88RClfBXgFMpxS5Pb4OgzXIn4sQGkfBDo54lRgA
Wrqwt4JQGAv6EYdKBAA3O/uD/LkIoXEU7OAQcFlyCbe5A4MDoTBGwSrnCSqxvsviwzWQEQpbFARH
vErsdHu7DLgwB0LhioojDrUYAJo6zMH/aITQuKAsOG60Y3AgFK4oCA4eh6mS8ZvacXwUoXAV1Cnn
AYkaSUVth9vjZTLoY3sHhBCFgj3l3C9JIymvaW/psiTHRj3gWyGEgo+am04naiSAwxwIhS1qgiMu
Rsik03CYA6EwRU1wMOg0rUrUhEccCIUnaoIDAJI0kh6T3WJzUlUAQmjMKAuOKXFRAHC9DQ86EAo/
lAWH/3zK9VYTVQUghMaMsuCQijhSEacRgwOhMERZcABAilba3NHv9ngprAEhNAbUBkeUx+vDtTkQ
CjvUTDn3S4mLAoDGVlOKVvog74MQCjJqppz7xUYL+RxmQ4tx5bykcXlDhFBwUNlVIQgiRRfVqDf6
fLgaGELhhMrgAIA0nczh8uLaxQiFF4qDI1UnBYD6FiO1ZSCERoXi4IhTinhsBgYHQuFlKDh8Pt8d
m0mSJCdyOXIaQaTopI16Iy56jlAYoQFAcXGxWq1OTk6urKwc3vbBBx9kZGRkZmZ+8MEHE1dBqk46
6PS0dOK93RAKG4ySkpJjx47V19cfOHBg69atZWVl/gaHw/Hzn/+8srKSRqPl5eWtW7eOzWZPRAXp
8TIAqLvZ51/EGCEU+milpaUFBQUSiWTTpk1VVVVO59B17gRBkCTZ1NTU0tLicDi83omaGB6nFPG5
zCvNhgl6f4TQuGPo9frc3FwAoNPpEomkp6cnLi4OAJhM5sqVKx9//HEAWLt2LYfDCbwmMHMUxmMy
GI0g0uNll671eLw+Bp3iwVqE0P1giEQiu93u/8Fut8vlcv/jkpKSc+fOdXR0AEBOTs7x48eXLVvm
bxqvmaMBGYny83VdjXpTeoJsfN8ZITQRaPn5+V9++aXb7T516pRGo+FyuSaTyWq1SiQSHo9HkiSN
RqPT6cOPOMZdZpIcAGqbeifuIxBC44ixZs2a8vLy9PR0l8u1b98+ACgqKsrKynrhhReWLVs2e/Zs
h8OxfPnyRYsWTVwRCgkvOopXe8OwdunEfQhCaNwQ/mkaNpuNx+MRBDGi2eVyAQCLxQpsKSwsHPeu
CgDsPXrl68qW//7ZUrFgQs7dIITG0dBgJJ/Pvz01AIDFYg1PjYmTPUUBAJdvYG8FoTAQKmcx0uJl
TDrt0jUMDoTCQKgEB4tJT42X1jb14iX2CIW+UAkOAMhKjrY7PNfbcPlihEJdCAWHf5ij5jr2VhAK
dSEUHDFSfoyUV32th+pCEEL3QOVixbfLSYn5qqK5r39QJuaO7zsjhMYRlYsV325GSvRXFc0XGrq/
lx8/QR+BEHpwIdRVAYAUrZTPYV5o6Ka6EITQdwmt4KDRiJyU6PqbRtugm+paEEJ3FVrBAQAzU2N8
JIlDpAiFspALjmlJChaTfv5qJ9WFIITuKuSCg8Wk50yJvnzDMOjE3gpCISrkggMA8tKVHq/vYiP2
VhAKUaEYHNlTolkM2rfYW0EoVIVicLCZ9OyU6NrrhkGnh+paEEJ3EIrBAQD56Sq313exESd0IBSK
QmvKeUD2lGg2k15R2zF3mmaCPgIhNGahNeU8gM2kz0yNOXelc8DuEvKCsQQZQuj+hWhXBQDmTNN4
feS3V3CIFKGQE7rBkZkoF/FZ5ZfbqS4EITRS6AYHjUbMzlTfaDN39VmprgUh9E9CNzgAYEFOLAB8
U40HHQiFlpAODm2MSKsUlde0+0hcwRihEBLSwQEAC7JjTQOOK014L3uEQkioB8ecaWo6jTh9oZXq
QhBCt4R6cAh5rLx0VVV9t3nAQXUtCKEhoR4cALA0V+sjydMX8aADoVARolPOh0vRSjUKwamq1h/M
T6bR7nCDW4RQkIXolPMRlubq9v79SnVjz4zUmCB/NELodmHQVQGAuVlqNpP+dVUL1YUghADCJTi4
bObcaeraG4Zuo43qWhBCYRIcALAkVwcAp6r0VBeCEAqf4NApRUmxktKLbS63l+paEJrshoLD5/Pd
7Rnf0RRky2bF2xzushq8dAUhitEAoLi4WK1WJycnV1ZWDm87dOhQfHx8YmLiq6++SlF5/yQ3TSkV
cb4820zipSsIUYpWUlJy7Nix+vr64uLirVu3BhqsVmthYeFnn31WU1Ozc+fOlhbqz2jQabRls+K7
jbaLDXjnBISoRCstLS0oKJBIJJs2baqqqnI6nf6Gb775Jjs722azlZaWXr16VavVUluo36IZWi6b
8bczN6guBKFJjabX6xUKBQDQ6XSJRNLTM/THvK2t7ezZs9u3bz98+HBubq7XGxJDklw24+E83Y02
c/3NPqprQWjyYohEIrvd7v/BbrfL5XL/Yx6Pp9VqP/74YwBYvHjx0aNHV61a5W8KTDkHKmaR/sus
+C8rmr8ou5EaLwvyRyOE/Bj5+fm7d+9+5plnysrKN

----------

## Clock face

In [11]:
my $dt = now.DateTime.Str.substr(0,19);
say $dt;
$cmd = '
p=ClockGauge["' ~ $dt ~ '"];
ps=ExportString[p,{"Base64","PNG"}]; 
Export["/tmp/wlimg.txt",ps]';
$wlRes = $wlProc.evaluate($cmd);
my $timg=slurp('/tmp/wlimg.txt');
from-base64($timg)

2022-12-24T19:54:19


-------

## Geo computations

Here we find the nearest ocean to our current location:

In [12]:
$cmd = 'GeoNearest["Ocean", Here]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n {$wlRes.raku}";

Sent : GeoNearest["Ocean", Here]
Got  :
 "\{Entity[Ocean, AtlanticOcean]}"


Here we find the Geo-coordinates of Miami, FL, USA and assign the results to the WL variable `loc`:

In [13]:
$cmd = 'loc = FindGeoLocation["Miami, FL"]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n {$wlRes.raku}";

Sent : loc = FindGeoLocation["Miami, FL"]
Got  :
 "GeoPosition[\{25.7743, -80.1937}]"


Here we find the cities with 30 miles of the location found above and convert the corresponding distance into kilometers:

In [14]:
$cmd = 'res = GeoNearest["City", loc, {7, Quantity[30, "Miles"]}]; res2 = Map[{#["Name"], QuantityMagnitude@UnitConvert[GeoDistance[loc, #],"Kilometers"]}&, res]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent   : $cmd";
say "Got    :\n {$wlRes.raku}";

Sent   : res = GeoNearest["City", loc, {7, Quantity[30, "Miles"]}]; res2 = Map[{#["Name"], QuantityMagnitude@UnitConvert[GeoDistance[loc, #],"Kilometers"]}&, res]
Got    :
 "\{\{Miami, 0.}, \{Miami Beach, 2.52844}, \{Fisher Island, 4.61465}, \{Brownsville, 5.43461}, \{Coral Gables, 6.07561}, \{Gladeview, 7.10614}, \{Miami Springs, 7.38113}}"


In [16]:
$cmd = 'p=GeoGraphics[GeoMarker/@GeoNearest["City", loc, {7, Quantity[30, "Miles"]}]];ps=ExportString[p,{"Base64","PNG"}]; Export["/tmp/wlimg.txt",ps]';
$wlRes = $wlProc.evaluate($cmd);
my $timg=slurp('/tmp/wlimg.txt');
from-base64($timg)

-------

## Using Wolfram|Alpha

Here we use [Wolfram|Alpha](https://www.wolframalpha.com) to find the distance from Earth to Sun:

In [17]:
$cmd = 'WolframAlpha["Earth distance from Sun", "Result"]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n {$wlRes.raku}";

Sent : WolframAlpha["Earth distance from Sun", "Result"]
Got  :
 "0.983621 astronomical units"


Here we find the dietary [callories in 20 servings of potato salad](https://www.wolframalpha.com/input?i=calories+in+20+servings+of+potato+salad):

In [19]:
$cmd = 'WolframAlpha["calories in 20 servings of ice cream", "Result"]';
$wlRes = $wlProc.evaluate($cmd);
say "Sent : $cmd";
say "Got  :\n {$wlRes.raku}";

Sent : WolframAlpha["calories in 20 servings of ice cream", "Result"]
Got  :
 "3200. dietary Calories"


In [20]:
$cmd = 'p=WolframAlpha["calories in 20 servings of potato salad", {{"NutritionLabelSingle:ExpandedFoodData", 1}, "Image"}]; ps=ExportString[p,{"Base64","PNG"}]; Export["/tmp/wlimg.txt",ps]';
$wlRes = $wlProc.evaluate($cmd);
$timg=slurp('/tmp/wlimg.txt');
from-base64($timg)

------

## Using repostiory functions

In [21]:
$cmd = '
p=ResourceFunction["RandomMondrian"][];
ps=ExportString[p,{"Base64","PNG"}]; 
Export["/tmp/wlimg.txt",ps]';
$wlRes = $wlProc.evaluate($cmd);
my $timg=slurp('/tmp/wlimg.txt');
from-base64($timg)

<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAkAAAAFiCAIAAACRdPDBAAAA1HpUWHRSYXcgcHJvZmlsZSB0eXBl
IGV4aWYAAHjadU9ZrgMhDPvnFD1CNhw4DtNhpHeDd/yGpaqmUi0RG+MEJfX/vys9BoQpWfaCClDA
qlVpIQotLGaqs04YkWz35ifIktyC8+cBfTEfd9+xWMrXoD2fdPwQWm037EEqy+e27sfOSyvlNug6
rom9DNl7Lf5xVxdksFtUE3JHDV1SyCBkoey48AQiyDiF7Bnc4R5t8JHQNhoHpPs93RM0jHOsoOOI
NuVZReO/0BY6q0/H5YxUDk061nkBaHZcQUmUEq4AAAAJcEhZcwAACxMAAAsTAQCanBgAAAA8dEVY
dFNvZnR3YXJlAENyZWF0ZWQgd2l0aCB0aGUgV29sZnJhbSBMYW5ndWFnZSA6IHd3dy53b2xmcmFt
LmNvbVyipoUAAAAhdEVYdENyZWF0aW9uIFRpbWUAMjAyMjoxMjoyNCAxNDo1NzoyN4/rJgUAAAiG
SURBVHic7dxNceNYGEBRvSmjMI4QCIDgCIGG0QSCIwBCIDhMQ7OYjVszUTvjH/m2z9mprFJ9smzf
UvQqY57nCQBq/tp6AAD4PwQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABI2q28Nsa42RwAsLD+rzbc
gQGQJGAAJAkYAElrz8AW7vPf/i4e1O1f3raa5MYO76/Hm/d54sUhL/I5X3ws7/OYNxjy/Cu+uDof
0/7MA07T9DwdjjcTv2z3OeQ1fGvthTswAJIEDIAkAQMgScAASBIwAJIEDIAkAQMgScAASBIwAJIE
DIAkAQMgScAASBIwAJIEDIAkAQMgScAASBIwAJIEDIAkAQMgScAASBIwAJIEDICk3em7jjGuNweP
4PD+uvUIv3eNz3nlmPfveTpc/JiP+U7+GdyBAZAkYAAkCRgASQIGQNI3FnHM83y9Of43D2BD9i9v
W4/wHxZLSy7yOV98LC9y4hefs/jduco7+Xn+IS9vPG09QYE7MACSBAyAJAEDIEnAAEgSMACSBAyA
JAEDIEnAAEgSMACSBAyAJAEDIEnAAEgSMACSBAyAJAEDIEnAAEgSMACSBAyAJAEDIEnAAEgSMACS
BAyApDHP85evjXHLUS5i//K29Qg3cnh/Pd68zxNfDAnwLSuFmtyBARAlYAAkCRgASQIGQNLu9F3X
H6ZtpbjS5GHNn1tPcCvjaesJvi+xDugiv0KLH42PaX/+MZ+nw/Hm+XMuhrzPq3MN31r55Q4MgCQB
AyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQM
gCQBAyBJwABIEjAAkgQMgKQxz/OXr41xvLmy54YWQwJ8y8e0P/8gz9PhePP8X0u/bP9YfyfdgQGQ
JGAAJAkYAEkCBkDSbusBLuw+V5o8rMWD6Is8LU9YPNLfv7xtNcmKw/vr8eZFvjsXv+IXXxwxNddH
zJ9bT3Ar4+kbO7sDAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQB
AyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIGvM8f/naGLcc5SJWTofbK36EHtZF
vjuLK/4x7c884PN0ON68xpDcs/Ur7g4MgCQBAyBJwABIEjAAknan73r+89hrWDzj5Z49zhKbxTKB
ayw9OP+Y1jL8I3F17vO7c4PP+Tp3YAAkCRgASQIGQJKAAZAkYAAkCRgASQIGQJKAAZAkYAAkCRgA
SQIGQJKAAZAkYAAkCRgASQIGQJKAAZAkYAAkCRgASQIGQJKAAZAkYAAkCRgASWOe5y9fG+OWowDA
sZVCTe7AAIgSMACSBAyApN3pu67/LZIVi6eJ8+dWg/zGePpl8z6v+MXfzMRZT/8+8buf8yITVs76
4q5x4okfosX3cd03AgbwW4/TGDbnT4gAJAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAAJAkY
AEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAA
JAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQ
JGAAJAkYAEkCBkCSgAGQJGAAJAkYAEkCBkCSgAGQJGAAJO22HgDgVGOMrUfYxjVOfDxd/JC35g4M
gCQBAyBJwABIEjAAkiziADLmz60n+MJiQcQ8z+ce8NdVG+cf8BoWQ+5f3s4/5uH99fSd3YEBkCRg
ACQJGABJAgZAkoABkCRgACQJGABJAgZAkoABkCRgACQJGABJAgZAkoABkCRgACQJGABJAgZAkoAB
kCRgACQJGABJAgZAkoABkDTmef7ytTFuOcrjmD+3nuAL42nrCWBV5buz8rt66gH9/E7T9Lt30h0Y
AEkCBkCSgAGQJGAAJO1O3/X8x5IPK/o81hX/s0U/lo/pIl/GxRW/zxUx31pK5g4MgCQBAyBJwABI
EjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJ
wABIEjAAkgQMgCQBAyBpzPP85WtjHG+u7Mm6xTtZ4Yr/2aIfSx7K+q+QOzAAkgQMgCQBAyBJwABI
2m09wCO628URnupz5yrfnY9pv9UkK56nw/Hm+W/m4qx//vx55gGnafrx48fpO7sDAyBJwABIEjAA
kgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABI
EjAAkgQMgCQBAyBJwABIGvM8f/naGMebK3uyrvJOLuYE2Nb6r6U7MACSBAyAJAEDIGm39QDcr/3L
29YjVB3eX4837/ap5/1bPJf9mPZbTbLueTocb97nFb/4m3mNs/7Wk3h3YAAkCRgASQIGQJKAAZAk
YAAkCRgASQIGQJKAAZAkYAAkCRgASQIGQJKAAZAkYAAkCRgASQIGQJKAAZAkYAAkCRgASQIGQJKA
AZAkYAAkCRgASbutBwA41fN02HqEk4wxth7h9y7+Zt7+rN2BAZAkYAAkCRgASQIGQJJFHEDG/uVt
6xH+2+H99Xhz/jz3gOPpl82PaX/uEa9gsQxknufzj/mtlSDuwABIEjAAkgQMgCQBAyBJwABIEjAA
kgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABIEjAAkgQMgCQBAyBJwABI
EjAAksY8z1++NsYtRwGAYyuFmtyBARAlYAAkCRgASQIGQNLaIg4AuFvuwABIEjAAkgQMgCQBAyBJ
wABIEjAAkgQMgCQBAyDpbxjSPI4zHLh2AAAAAElFTkSuQmCC
">

In [23]:
$cmd = '
p=ResourceFunction["RandomMandala"]["Radius"->{10,8,4},"ConnectingFunction"->FilledCurve@*BezierCurve,ColorFunction -> "Rainbow",Background->Automatic];
ps=ExportString[p,{"Base64","PNG"}]; 
Export["/tmp/wlimg.txt",ps]';
$wlRes = $wlProc.evaluate($cmd);
my $timg=slurp('/tmp/wlimg.txt');
from-base64($timg)

-----

## Epilog

Here we close the ZMQ connection:

In [ ]:
#$wlProc.terminate